Having recently completed [deeplearning.ai](https://www.coursera.org/specializations/deep-learning)'s I want to continue to build and learn the models. The courses were great, I have a much better understanding about Neural Network architecture as a result of taking them. The programming exercises were good and I feel the course is stuck 
in between a rock and a hard place. They could either make the programming exercises such that coded needed to be 
generated from scratch, or they could mock up a skeleon of the program and leave the details to the student. They favored doing the later implementation. As a result when I've gone to sit down and right my own program I'm left scratching my head. 

When you Google how to do certain things in tensorflow, for example how to get started with writing a program many things surface, the [TensorFlow](https://www.tensorflow.org/) website first among them. The documentation there is 
great, however when you already know a little about TensorFlow and building Neural Networks, and Google armed with that information, you can find yourself in documentation that you don't want to use.

### Mini-batch Processing

A cool concept in fitting Neural Network models is Mini-batch processing. Briefly, rather than using every datapoint in every iteration of feed-forward and back propagation, a smaller subset of the data is used.

The idea being that the steps taken during any one iteration may harm the model, in general you will decrease your loss function much more quickly. A surprising thing to me about mini-batch processing is that the sizes of the batches are quite small, compared to the data size as well as my expectations.

I would have expected the batches to be on the order of 50% or 10% or 5% of the number of training examples. In practice, however the batch sizes are quite small. Batch sizes of 64, 128, 256 or 512 are quite common. For a dataset with one million training examples that represents a batch size of less than one tenth of one percent.

### Data

For this data I'll generate some fake data to illustrate the concepts. We'll use 1000 observations, three features (x1, x2, x3) and and output (y) that has a linear relationship with the features.

In [1]:
import numpy as np
# Set a seed
np.random.seed(100)
# Create three variables, x1, x2 and x3
x1 = 10*np.random.rand(1000)
x2 = -3*np.random.rand(1000)
x3 = 2*np.random.rand(1000)
# make y a linear combination of these variables, plus some noise
y = 11 + 1.5*x1 + 6*x2 -3*x3 + np.random.normal(loc = 0, scale = 3, size = 1000)
y = y.reshape(1000,1)
data_dict = {"y": y,
            "x1": x1,
            "x2": x2,
            "x3": x3}

### High Level API: Estimators

We'll follow the recommended program structure for using pre-made estimators from the [TensorFlow](https://www.tensorflow.org/programmers_guide/estimators) website. 

In [2]:
import tensorflow as tf

##### Write one or more dataset inputting functions

In [3]:
# The data pipleine for this problem is straightforward, so this function is trivial. 
def data_input(data_dict):
     
    features = {"x1": data_dict["x1"],
               "x2": data_dict["x2"],
               "x3": data_dict["x3"]}
    
    labels = data_dict["y"]
    
    return features, labels

features, labels = data_input(data_dict)

##### Define the feature columns

In [4]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in features.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[_NumericColumn(key='x1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='x2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='x3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


##### Instantiate the relevant pre-made Estimator

In [5]:
regressor = tf.estimator.LinearRegressor(feature_columns = my_feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplxb81ko9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19e2144908>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### Call a training, evaluation, or inference method.

In [6]:
regressor.train(input_fn=lambda:data_input(data_dict),steps = 2)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplxb81ko9/model.ckpt.
INFO:tensorflow:loss = 109605.75, step = 1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/tmplxb81ko9/model.ckpt.
INFO:tensorflow:Loss for final step: 86529.3.


### Try a different Approach:
#### Import data using Dataset API
#### Fit model using Estimator API
###### This is what the TensorFlow documents suggest!!

In [7]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
features = {x_key: data_dict[x_key] for x_key in ["x1","x2","x3"]}
labels = data_dict["y"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features["x1"].shape[0] == labels.shape[0]

features_placeholder1 = tf.placeholder(features["x1"].dtype, features["x2"].shape)
features_placeholder2 = tf.placeholder(features["x2"].dtype, features["x2"].shape)
features_placeholder3 = tf.placeholder(features["x3"].dtype, features["x3"].shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder1,
                                              features_placeholder2,
                                              features_placeholder3,                                      
                                              labels_placeholder))
iterator = dataset.make_initializable_iterator()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={features_placeholder1: features["x1"],
                                          features_placeholder2: features["x2"],
                                          features_placeholder3: features["x3"],
                                          labels_placeholder: labels})

But this needs to be a function...

In [8]:
def data_input2(data_dict,num_epochs):
    features = {x_key: data_dict[x_key] for x_key in ["x1","x2","x3"]}
    labels = data_dict["y"]


    features_placeholder1 = tf.placeholder(features["x1"].dtype, features["x2"].shape)
    features_placeholder2 = tf.placeholder(features["x2"].dtype, features["x2"].shape)
    features_placeholder3 = tf.placeholder(features["x3"].dtype, features["x3"].shape)
    labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

    dataset = tf.data.Dataset.from_tensor_slices((features_placeholder1,
                                                  features_placeholder2,
                                                  features_placeholder3,                                      
                                                  labels_placeholder))
    dataset.repeat(num_epochs)
    #iterator = dataset.make_one_shot_iterator()
    
    #features, labels = iterator.get_next()
    #return features, labels
    return dataset
    

In [9]:
### Examine this more closely:
def dataset_input_fn():
#filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
#dataset = tf.data.TFRecordDataset(filenames)

  # Use `tf.parse_single_example()` to extract data from a `tf.Example`
  # protocol buffer, and perform any additional per-record preprocessing.
  def parser(record):
    keys_to_features = {
        "image_data": tf.FixedLenFeature((), tf.string, default_value=""),
        "date_time": tf.FixedLenFeature((), tf.int64, default_value=""),
        "label": tf.FixedLenFeature((), tf.int64,
                                    default_value=tf.zeros([], dtype=tf.int64)),
    }
    
    parsed = tf.parse_single_example(record, keys_to_features)

    #Perform additional preprocessing on the parsed data.
    #image = tf.image.decode_jpeg(parsed["image_data"])
    #image = tf.reshape(image, [299, 299, 1])
    #label = tf.cast(parsed["label"], tf.int32)

    return {"image_data": image, "date_time": parsed["date_time"]}, label

  # Use `Dataset.map()` to build a pair of a feature dictionary and a label
  # tensor for each example.
  dataset = dataset.map(parser)
  #dataset = dataset.shuffle(buffer_size=10000)
  dataset = dataset.batch(32)
  dataset = dataset.repeat(num_epochs)
  iterator = dataset.make_one_shot_iterator()

  # `features` is a dictionary in which each value is a batch of values for
  # that feature; `labels` is a batch of labels.
  features, labels = iterator.get_next()
  return features, labels

In [10]:
def data_input3(shape):

    features_placeholder1 = tf.placeholder(tf.float32, shape)
    features_placeholder2 = tf.placeholder(tf.float32, shape)
    features_placeholder3 = tf.placeholder(tf.float32, shape)
    labels_placeholder    = tf.placeholder(tf.float32, shape)

    dataset = tf.data.Dataset.from_tensor_slices((features_placeholder1,
                                                  features_placeholder2,
                                                  features_placeholder3,                                      
                                                  labels_placeholder))
    #def parser():
    #    labels = dataset["label"]
    #    features = {"x1": dataset["x1"],
    #                "x2": dataset["x2"],
    #                "x3": dataset["x3"]}
    
    #dataset = dataset.map(parser)    
    
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

#f, l = data_input3(x1.shape)

In [11]:
regressor2 = tf.estimator.LinearRegressor(feature_columns = features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpcghv82hz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19b4295e80>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
regressor2.train(input_fn=lambda:data_input2(data_dict,num_epochs = 1))

ValueError: features should be a dictionary of `Tensor`s. Given type: <class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>

So once again, I'm trying to combine features I can't.
I cannot use:
* Dataset API
* Placeholders
* Input_fn
* one_shot_iterator 

All together.

I still want to, for this example use:
* Dataset API
* Estimator API
* Dataset input_fun, to feed the estimator.

### Make an Input function that uses the dataset API, feed in the actual data, not placeholders.

In [13]:
def data_input4(data_dict):
    
    # Create the dataset
    
    
    
    
#     feature_dict = {x_key: tf.feature_column.numeric_column(key=x_key,
#                                                             dtype=tf.float32,
#                                                             default_value=tf.zeros([], dtype=tf.int64)
#                                                             shape = data_dict[x_key].shape)

#                     for x_key in ["x1","x2","x3"]}

#     features = {"x1": data_dict["x1"],
#                "x2": data_dict["x2"],
#                "x3": data_dict["x3"]}
    
    features = {"x1": tf.feature_column.numeric_column(key="x1"),
                "x2": tf.feature_column.numeric_column(key="x2"),
                "x3": tf.feature_column.numeric_column(key="x3")
               }
    
#     x1 = tf.feature_column.numeric_column(key="x1")
#     x2 = tf.feature_column.numeric_column(key="x2")
#     x3 = tf.feature_column.numeric_column(key="x3")
    
#     features = {"x1": x1,
#                 "x2": x2,
#                 "x3": x3}
    
    
    
#     my_feature_columns = []
#     for key in features.keys():
#         my_feature_columns.append(tf.feature_column.numeric_column(key=key))
                                                            
                                                            
                                                            

    #dataset = tf.data.Dataset.from_tensor_slices((dict(my_feature_columns),data_dict["y"]))
    dataset = tf.data.Dataset.from_tensor_slices((features,data_dict["y"]))   
    

#     dataset = tf.data.Dataset.from_tensor_slices((data_dict["x1"],
#                                                   data_dict["x2"],
#                                                   data_dict["x3"],
#                                                   data_dict["y"]))
 
  
    
#     # parse into a features dictionary and labels array
#     def parser(d1,d2,d3,d4):
        
#         keys_to_features = {
#         "x1":    tf.FixedLenFeature((),tf.float32, default_value=tf.zeros([], dtype=tf.float32)),
#         "x2":    tf.FixedLenFeature((),tf.float32, default_value=tf.zeros([], dtype=tf.float32)),
#         "x3":    tf.FixedLenFeature((),tf.float32, default_value=tf.zeros([], dtype=tf.float32)),
#         "label": tf.FixedLenFeature((),tf.float32,default_value=tf.zeros([], dtype=tf.float32))
#         } 
        
#         parsed = tf.parse_single_example("",keys_to_features)
        
#         x1 = parsed["x1"]
#         x2 = parsed["x2"]
#         x3 = parsed["x3"]
            

#         return {"x1": x1,"x2": x2,"x3": x3}, parsed["label"]

    
#     dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

In [14]:
data_input4(data_dict)

ValueError: None values not supported.

In [15]:
regressor4 = tf.estimator.LinearRegressor(feature_columns = features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpx_qx7vde', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19b418f1d0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
regressor4.train(input_fn=lambda:data_input4(data_dict), steps = 2)

ValueError: None values not supported.

In [17]:
## For Stack Overflow:

import numpy as np
import tensorflow as tf

np.random.seed(100)
x1 = np.random.rand(100)
x2 = np.random.rand(100)
x3 = np.random.rand(100)
y = np.random.rand(100)

data_dict = {"x1": x1,
                 "x2": x2,
                 "x3": x3,
                  "y": y}

features = {"x1": x1,
           "x2": x2,
           "x3": x3}

def input_fn(data_dict):
    features = {"x1": data_dict["x1"],
                "x2": data_dict["x2"],
                "x3": data_dict["x3"]}
    
#     features = {"x1": tf.feature_column.numeric_column(key="x1"),
#                 "x2": tf.feature_column.numeric_column(key="x2"),
#                 "x3": tf.feature_column.numeric_column(key="x3")
#                }
                   
    dataset = tf.data.Dataset.from_tensor_slices((features,data_dict["y"]))   
    
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

#regressor = tf.estimator.LinearRegressor(feature_columns = lambda: input_fn(data_dict)[0])
regressor = tf.estimator.LinearRegressor(feature_columns = features)
regressor.train(input_fn=lambda:input_fn(data_dict), steps = 1)

#I get the following error:
    
#    ValueError: Items of feature_columns must be a _FeatureColumn. Given (type <class 'str'>): x1.

#If I replace the `features` definition in `input_fn` by this:

#         features = {"x1": tf.feature_column.numeric_column(key="x1"),
#                 "x2": tf.feature_column.numeric_column(key="x2"),
#                 "x3": tf.feature_column.numeric_column(key="x3")
#                }
# And execute, I get this error:

#     ValueError: None values not supported.

# So my question is:

# Within the framework of the Dataset and Estimator APIs, how can I use numpy arrays as data, specify feature columns within the `input_fn` and then pass to an Estimator?

    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9giylo8t', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19b42b0c88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


ValueError: Items of feature_columns must be a _FeatureColumn. Given (type <class 'str'>): x1.

In [18]:
# https://stackoverflow.com/questions/49088781/tensorflow-valueerror-feature-key-c1-cannot-have-rank-0
import numpy as np
import tensorflow as tf

# create data
np.random.seed(100)
x1 = np.random.rand(100)
x2 = np.random.rand(100)
x3 = np.random.rand(100)
y = np.random.rand(100)
#


X1 = tf.feature_column.numeric_column(key="X1")
X2 = tf.feature_column.numeric_column(key="X2")
X3 = tf.feature_column.numeric_column(key="X3")
feature_columns = [X1,X2,X3]

features = {"X1": x1,
           "X2": x2,
           "X3": x3}


# data = tf.data.Dataset.from_tensor_slices({"feature_dict":features,
#                                            "labels": y})

# def input_fn(data):
    
#     iterator = data.make_one_shot_iterator()
#     feature_dict, labels = iterator.get_next()
#     return feature_dict, labels


def input_fn(features, labels):

    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Build the Iterator, and return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()



regressor = tf.estimator.LinearRegressor(feature_columns = feature_columns)

regressor.train(input_fn=lambda:input_fn(features,y), steps = 1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpomh2tcec', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f19b4235cf8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


ValueError: Feature (key: X1) cannot have rank 0. Give: Tensor("IteratorGetNext:0", shape=(), dtype=float64, device=/device:CPU:0)